# Query 2

## Import and describe data

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import * #col, when, count
import time

# Create SparkSession
spark = SparkSession.builder \
    .appName("Query2") \
    .config("spark.executor.instances", "4") \
    .getOrCreate()

# Load and filter Data
crime_data_path_1 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime_data_path_2 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"
data_1 = spark.read.csv(crime_data_path_1, header=True, inferSchema=True) \
    .withColumnRenamed('AREA ', 'AREA')
data_2 = spark.read.csv(crime_data_path_2, header=True, inferSchema=True)
data = (
    data_1.union(data_2)
    # Keep only selected columns
    .select('DR_NO', 'DATE OCC', 'AREA', 'AREA NAME', 'Status', 'Status Desc')
    # Convert DATE OCC to timestamp type
    .withColumn("DATE OCC", to_timestamp("DATE OCC", "MM/dd/yyyy hh:mm:ss a"))
)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2536,application_1732639283265_2495,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# View first lines of dataset (for validation)
data.limit(3).show()

# Describe the dataset
data.describe().show() 

# Print schema
data.printSchema()

# Get distinct values of some columns
data.select('Status', 'Status Desc').distinct().show()
data.select('AREA', 'AREA NAME').distinct().orderBy('AREA').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------+----+---------+------+------------+
|   DR_NO|           DATE OCC|AREA|AREA NAME|Status| Status Desc|
+--------+-------------------+----+---------+------+------------+
| 1307355|2010-02-20 00:00:00|  13|   Newton|    AA|Adult Arrest|
|11401303|2010-09-12 00:00:00|  14|  Pacific|    IC| Invest Cont|
|70309629|2010-08-09 00:00:00|  13|   Newton|    IC| Invest Cont|
+--------+-------------------+----+---------+------+------------+

+-------+--------------------+-----------------+-----------+-----------------+------------+
|summary|               DR_NO|             AREA|  AREA NAME|           Status| Status Desc|
+-------+--------------------+-----------------+-----------+-----------------+------------+
|  count|             3113337|          3113337|    3113337|          3113333|     3113337|
|   mean|1.7099427277162287E8|10.96417927130921|       NULL|             16.0|        NULL|
| stddev| 4.182187908303369E7|6.046025657331321|       NULL|4.242640687119285

## Query - DataFrames API

In [3]:
from pyspark.sql.window import Window

# Start timing
start_time = time.time()

# Load Data and filter data
crime_data_path_1 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime_data_path_2 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"
data_1 = spark.read.csv(crime_data_path_1, header=True, inferSchema=True) \
    .withColumnRenamed('AREA ', 'AREA')
data_2 = spark.read.csv(crime_data_path_2, header=True, inferSchema=True)
data = (
    data_1.union(data_2)
    # Drop null island (0,0) entries
#     .filter((col('LON') != 0) | (col('LAT') != 0))
    # Keep only selected columns
    .select('DR_NO', 'DATE OCC', 'AREA', 'AREA NAME', 'Status', 'Status Desc')
    # Convert DATE OCC to timestamp type
    .withColumn("DATE OCC", to_timestamp("DATE OCC", "MM/dd/yyyy hh:mm:ss a"))
)

# Add a "year" column
data = data.withColumn("year", year("DATE OCC"))

# Create a column indicating whether the case is open or closed
data = data.withColumn("open_closed", when(col("Status").isin("NULL", "IC"), "open").otherwise("closed"))

# Group by year and police station, and calculate closed cases and total cases
cases_grouped = data.groupBy("year", "AREA NAME").agg(  
    count("*").alias("total_cases"),
    sum(when(col("open_closed") == "closed", 1).otherwise(0)).alias("closed_cases")
)

# Calculate the closed cases rate
cases_grouped = cases_grouped.withColumn(
    "closed_rate", (col("closed_cases") / col("total_cases"))*100
)

# Rank the stations by closed cases rate within each year
window_spec = Window.partitionBy("year").orderBy(col("closed_rate").desc())

# Add a rank column
ranked_stations = cases_grouped.withColumn("rank", row_number().over(window_spec))

# Filter to get the top 3 stations per year
top_3_stations_per_year = ranked_stations.filter(col("rank") <= 3)

# Show the result
result = top_3_stations_per_year.select("year", "AREA NAME", "closed_rate", "rank")
result.show()

# Stop timing and print out the execution duration
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

# Save Results
output_path = "s3://groups-bucket-dblab-905418150721/group7/q2_results"
result.write \
    .option("header", True) \
    .mode("overwrite") \
    .csv(f"{output_path}/dataframes_csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+----+
|year|  AREA NAME|       closed_rate|rank|
+----+-----------+------------------+----+
|2010|    Rampart| 32.84713448949121|   1|
|2010|    Olympic|31.515289821999087|   2|
|2010|     Harbor| 29.36028339237341|   3|
|2011|    Olympic|35.040060090135206|   1|
|2011|    Rampart|  32.4964471814306|   2|
|2011|     Harbor| 28.51336246316431|   3|
|2012|    Olympic| 34.29708533302119|   1|
|2012|    Rampart| 32.46000463714352|   2|
|2012|     Harbor|29.509585848956675|   3|
|2013|    Olympic| 33.58217940999398|   1|
|2013|    Rampart|  32.1060382916053|   2|
|2013|     Harbor|29.735499940695053|   3|
|2014|   Van Nuys|  32.0215235281705|   1|
|2014|West Valley| 31.49754809505847|   2|
|2014|    Mission|31.224939855653567|   3|
|2015|   Van Nuys|32.265140677157845|   1|
|2015|    Mission|30.463762673676303|   2|
|2015|   Foothill|30.353001803658852|   3|
|2016|   Van Nuys|32.194518462124094|   1|
|2016|West Valley| 31.40146437042384|   2|
+----+-----

## Query - SQL API

In [4]:
# Start timing
start_time = time.time()

# Load Data and modify data
crime_data_path_1 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime_data_path_2 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"

data_1 = spark.read.csv(crime_data_path_1, header=True, inferSchema=True) \
    .withColumnRenamed('AREA ', 'AREA')
data_2 = spark.read.csv(crime_data_path_2, header=True, inferSchema=True)

data_1.createOrReplaceTempView("data_1")
data_2.createOrReplaceTempView("data_2")
data = spark.sql("""
    SELECT 
        DR_NO, 
        TO_TIMESTAMP(`DATE OCC`, 'MM/dd/yyyy hh:mm:ss a') AS `DATE OCC`, 
        AREA, 
        `AREA NAME`, 
        Status, 
        `Status Desc`
    FROM (
        SELECT * FROM data_1
        UNION ALL
        SELECT * FROM data_2
    )
""")

# Create a temporary view for SQL queries
data.createOrReplaceTempView("crime_data")

# SQL Query to add "year" column and determine "open_closed" status
query = """
    SELECT 
        *, 
        YEAR(`DATE OCC`) AS year,
        CASE 
            WHEN Status IN ('NULL', 'IC') THEN 'open' 
            ELSE 'closed' 
        END AS open_closed
    FROM crime_data
"""
data_with_columns = spark.sql(query)
data_with_columns.createOrReplaceTempView("crime_data_with_columns")

# SQL Query to group by year and police station, calculate closed cases and total cases
group_query = """
    SELECT 
        year, 
        `AREA NAME`,
        COUNT(*) AS total_cases,
        SUM(CASE WHEN open_closed = 'closed' THEN 1 ELSE 0 END) AS closed_cases,
        100 * SUM(CASE WHEN open_closed = 'closed' THEN 1 ELSE 0 END) / COUNT(*) AS closed_rate
    FROM crime_data_with_columns
    GROUP BY year, `AREA NAME`
"""
cases_grouped = spark.sql(group_query)
cases_grouped.createOrReplaceTempView("cases_grouped")

# SQL Query to rank stations by closed rate within each year
rank_query = """
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY year ORDER BY closed_rate DESC) AS rank
    FROM cases_grouped
"""
ranked_stations = spark.sql(rank_query)
ranked_stations.createOrReplaceTempView("ranked_stations")

# SQL Query to filter the top 3 stations per year
top_3_query = """
    SELECT 
        year, 
        `AREA NAME`, 
        closed_rate, 
        rank
    FROM ranked_stations
    WHERE rank <= 3
"""
top_3_stations_per_year = spark.sql(top_3_query)

# Show the result
result = top_3_stations_per_year.select("year", "AREA NAME", "closed_rate", "rank")
result.show()

# Stop timing and print out the execution duration
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

# Save Results
output_path = "s3://groups-bucket-dblab-905418150721/group7/q2_results"
result.write \
    .option("header", True) \
    .mode("overwrite") \
    .csv(f"{output_path}/sql_csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+----+
|year|  AREA NAME|       closed_rate|rank|
+----+-----------+------------------+----+
|2010|    Rampart| 32.84713448949121|   1|
|2010|    Olympic|31.515289821999087|   2|
|2010|     Harbor| 29.36028339237341|   3|
|2011|    Olympic|  35.0400600901352|   1|
|2011|    Rampart|32.496447181430604|   2|
|2011|     Harbor|28.513362463164313|   3|
|2012|    Olympic| 34.29708533302119|   1|
|2012|    Rampart| 32.46000463714352|   2|
|2012|     Harbor| 29.50958584895668|   3|
|2013|    Olympic| 33.58217940999398|   1|
|2013|    Rampart|  32.1060382916053|   2|
|2013|     Harbor|29.735499940695053|   3|
|2014|   Van Nuys|  32.0215235281705|   1|
|2014|West Valley| 31.49754809505847|   2|
|2014|    Mission| 31.22493985565357|   3|
|2015|   Van Nuys|32.265140677157845|   1|
|2015|    Mission|30.463762673676303|   2|
|2015|   Foothill|30.353001803658852|   3|
|2016|   Van Nuys|32.194518462124094|   1|
|2016|West Valley| 31.40146437042384|   2|
+----+-----

## Why is SQL faster than DataFrames API?
- DataFrames API is more verbose => more operations
- Declarative structure of SQL

## Convert data to .parquet format
And store to bucket

In [5]:
csv_path_1 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
csv_path_2 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"
parquet_path_1 = "s3://groups-bucket-dblab-905418150721/group7/q2_parquets/data_1/"
parquet_path_2 = "s3://groups-bucket-dblab-905418150721/group7/q2_parquets/data_2/"

data_1 = spark.read.csv(csv_path_1, header=True, inferSchema=True)
data_2 = spark.read.csv(csv_path_2, header=True, inferSchema=True)

data_1.write.parquet(parquet_path_1, mode="overwrite")
data_2.write.parquet(parquet_path_2, mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Test SQL+parquet

In [6]:
# Start timing
start_time = time.time()

# Load Data and modify data
crime_data_path_1 = "s3://groups-bucket-dblab-905418150721/group7/q2_parquets/data_1/"
crime_data_path_2 = "s3://groups-bucket-dblab-905418150721/group7/q2_parquets/data_2/"

data_1 = spark.read.parquet(crime_data_path_1, header=True, inferSchema=True) \
    .withColumnRenamed('AREA ', 'AREA')
data_2 = spark.read.parquet(crime_data_path_2, header=True, inferSchema=True)

data_1.createOrReplaceTempView("data_1")
data_2.createOrReplaceTempView("data_2")
data = spark.sql("""
    SELECT 
        DR_NO, 
        TO_TIMESTAMP(`DATE OCC`, 'MM/dd/yyyy hh:mm:ss a') AS `DATE OCC`, 
        AREA, 
        `AREA NAME`, 
        Status, 
        `Status Desc`
    FROM (
        SELECT * FROM data_1
        UNION ALL
        SELECT * FROM data_2
    )
""")

# Create a temporary view for SQL queries
data.createOrReplaceTempView("crime_data")

# SQL Query to add "year" column and determine "open_closed" status
query = """
    SELECT 
        *, 
        YEAR(`DATE OCC`) AS year,
        CASE 
            WHEN Status IN ('NULL', 'IC') THEN 'open' 
            ELSE 'closed' 
        END AS open_closed
    FROM crime_data
"""
data_with_columns = spark.sql(query)
data_with_columns.createOrReplaceTempView("crime_data_with_columns")

# SQL Query to group by year and police station, calculate closed cases and total cases
group_query = """
    SELECT 
        year, 
        `AREA NAME`,
        COUNT(*) AS total_cases,
        SUM(CASE WHEN open_closed = 'closed' THEN 1 ELSE 0 END) AS closed_cases,
        100 * SUM(CASE WHEN open_closed = 'closed' THEN 1 ELSE 0 END) / COUNT(*) AS closed_rate
    FROM crime_data_with_columns
    GROUP BY year, `AREA NAME`
"""
cases_grouped = spark.sql(group_query)
cases_grouped.createOrReplaceTempView("cases_grouped")

# SQL Query to rank stations by closed rate within each year
rank_query = """
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY year ORDER BY closed_rate DESC) AS rank
    FROM cases_grouped
"""
ranked_stations = spark.sql(rank_query)
ranked_stations.createOrReplaceTempView("ranked_stations")

# SQL Query to filter the top 3 stations per year
top_3_query = """
    SELECT 
        year, 
        `AREA NAME`, 
        closed_rate, 
        rank
    FROM ranked_stations
    WHERE rank <= 3
"""
top_3_stations_per_year = spark.sql(top_3_query)

# Show the result
result = top_3_stations_per_year.select("year", "AREA NAME", "closed_rate", "rank")
result.show()

# Stop timing and print out the execution duration
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

# Save Results
output_path = "s3://groups-bucket-dblab-905418150721/group7/q2_results"
result.write \
    .option("header", True) \
    .mode("overwrite") \
    .csv(f"{output_path}/sql_parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+----+
|year|  AREA NAME|       closed_rate|rank|
+----+-----------+------------------+----+
|2010|    Rampart| 32.84713448949121|   1|
|2010|    Olympic|31.515289821999087|   2|
|2010|     Harbor| 29.36028339237341|   3|
|2011|    Olympic|  35.0400600901352|   1|
|2011|    Rampart|32.496447181430604|   2|
|2011|     Harbor|28.513362463164313|   3|
|2012|    Olympic| 34.29708533302119|   1|
|2012|    Rampart| 32.46000463714352|   2|
|2012|     Harbor| 29.50958584895668|   3|
|2013|    Olympic| 33.58217940999398|   1|
|2013|    Rampart|  32.1060382916053|   2|
|2013|     Harbor|29.735499940695053|   3|
|2014|   Van Nuys|  32.0215235281705|   1|
|2014|West Valley| 31.49754809505847|   2|
|2014|    Mission| 31.22493985565357|   3|
|2015|   Van Nuys|32.265140677157845|   1|
|2015|    Mission|30.463762673676303|   2|
|2015|   Foothill|30.353001803658852|   3|
|2016|   Van Nuys|32.194518462124094|   1|
|2016|West Valley| 31.40146437042384|   2|
+----+-----

In [7]:
# spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…